# Big Data Project

Book Crossing DataSet <br>
** October 2021 ** <br>
<br>
<div class="alert alert-block alert-warning">
<b>PRESNTED BY:  </b> 
<strong>Or Shukrun  |  Mary Azencot |  Luba Zameslov <strong>
</div>
<br> <hr class="dotted"> <br>

### Apache Spark and Collab installation

  1) Install Java, Spark

  2) Set Environment Variables

  3) Start Spark Session

In [ ]:
!apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null 
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz 
!tar xf spark-3.1.2-bin-hadoop3.2.tgz 
!pip install -q findspark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,431 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http:

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init("spark-3.1.2-bin-hadoop3.2") 

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark.sql import Row
from pyspark.sql import functions

</div>
<br> <hr class="dotted"> <br>

### Establishing connection between "Colab" to "Google Drive"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/MyDrive/BD Project"   ### change the files path to your own device

Big_Data_Analysis_Project.ipynb  BX-Books.csv
BX-Book-Ratings.csv		 BX-Users.csv


</div>
<br> <hr class="dotted"> <br>

# Importing data to notebook

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests

import pyspark
from pyspark.sql import SparkSession

In [ ]:
#Users table
users = spark.read.csv('/content/drive/MyDrive/BD Project/BX-Users.csv', inferSchema = True, header = True,sep='"')

#Books table
items = spark.read.csv('/content/drive/MyDrive/BD Project/BX-Books.csv', inferSchema = True, header = True,sep='"')

#Ratings table
ratings = spark.read.csv('/content/drive/MyDrive/BD Project/BX-Book-Ratings.csv', inferSchema = True, header = True,sep='"')

In [ ]:
users.show(5)

+----+-------+---+--------------------+-----+----+----+
| _c0|User-ID| ;2|            Location|   ;4| Age|   "|
+----+-------+---+--------------------+-----+----+----+
|null|      1|  ;|  nyc, new york, usa|;NULL|null|null|
|null|      2|  ;|stockton, califor...|    ;|  18|   "|
|null|      3|  ;|moscow, yukon ter...|;NULL|null|null|
|null|      4|  ;|porto, v.n.gaia, ...|    ;|  17|   "|
|null|      5|  ;|farnborough, hant...|;NULL|null|null|
+----+-------+---+--------------------+-----+----+----+
only showing top 5 rows



In [ ]:
items.show(5)

+----+----------+---+--------------------+---+--------------------+---+-------------------+---+--------------------+---+--------------------+---+--------------------+---+--------------------+---+
| _c0|      ISBN| ;2|          Book-Title| ;4|         Book-Author| ;6|Year-Of-Publication| ;8|           Publisher|;10|         Image-URL-S|;12|         Image-URL-M|;14|         Image-URL-L|  "|
+----+----------+---+--------------------+---+--------------------+---+-------------------+---+--------------------+---+--------------------+---+--------------------+---+--------------------+---+
|null|0195153448|  ;| Classical Mythology|  ;|  Mark P. O. Morford|  ;|               2002|  ;|Oxford University...|  ;|http://images.ama...|  ;|http://images.ama...|  ;|http://images.ama...|  "|
|null|0002005018|  ;|        Clara Callan|  ;|Richard Bruce Wright|  ;|               2001|  ;|HarperFlamingo Ca...|  ;|http://images.ama...|  ;|http://images.ama...|  ;|http://images.ama...|  "|
|null|0060973129|  ;

In [ ]:
ratings.show(5)

+----+--------+----+----------+----+---+----+-----------+----+----+---+
| _c0|User-ID;| _c2|      ISBN| _c4|  ;| _c6|Book-Rating| _c8| _c9| ,,|
+----+--------+----+----------+----+---+----+-----------+----+----+---+
|null| 276725;|null|034545104X|null|  ;|null|          0|null|null| ,,|
|null| 276726;|null|0155061224|null|  ;|null|          5|null|null| ,,|
|null| 276727;|null|0446520802|null|  ;|null|          0|null|null| ,,|
|null| 276729;|null|052165615X|null|  ;|null|          3|null|null| ,,|
|null| 276729;|null|0521795028|null|  ;|null|          6|null|null| ,,|
+----+--------+----+----------+----+---+----+-----------+----+----+---+
only showing top 5 rows



# Cleaning data 

### Ratings Table

In [ ]:
from pyspark.sql.functions import *

In [ ]:
ratings = ratings.withColumn('User-ID;', regexp_replace('User-ID;', ';', '')).withColumnRenamed('User-ID;', 'User-ID')
df_Rating=ratings.select(['User-ID','ISBN','Book-Rating'])
df_Rating.show()

+-------+----------+-----------+
|User-ID|      ISBN|Book-Rating|
+-------+----------+-----------+
| 276725|034545104X|          0|
| 276726|0155061224|          5|
| 276727|0446520802|          0|
| 276729|052165615X|          3|
| 276729|0521795028|          6|
| 276733|2080674722|          0|
| 276736|3257224281|          8|
| 276737|0600570967|          6|
| 276744|038550120X|          7|
| 276745| 342310538|         10|
| 276746|0425115801|          0|
| 276746|0449006522|          0|
| 276746|0553561618|          0|
| 276746|055356451X|          0|
| 276746|0786013990|          0|
| 276746|0786014512|          0|
| 276747|0060517794|          9|
| 276747|0451192001|          0|
| 276747|0609801279|          0|
| 276747|0671537458|          9|
+-------+----------+-----------+
only showing top 20 rows



In [ ]:
df_Rating.printSchema()

root
 |-- User-ID: string (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- Book-Rating: string (nullable = true)



In [ ]:
from pyspark.sql.types import IntegerType

df_Rating = df_Rating.withColumn("Book-Rating",df_Rating["Book-Rating"].cast(IntegerType()))
df_Rating = df_Rating.withColumn("User-ID",df_Rating["User-ID"].cast(IntegerType()))
df_Rating.printSchema()

root
 |-- User-ID: integer (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- Book-Rating: integer (nullable = true)



Looking for NULL values within our columns

In [ ]:
from pyspark.sql.functions import col,isnan, when, count
df_Rating.select([count(when(isnan(N) | col(N).isNull(), N)).alias(N) for N in df_Rating.columns]).show()

+-------+----+-----------+
|User-ID|ISBN|Book-Rating|
+-------+----+-----------+
|      0|   1|         15|
+-------+----+-----------+



In [ ]:
df_Rating=df_Rating.dropna()
df_Rating.select([count(when(isnan(N) | col(N).isNull(), N)).alias(N) for N in df_Rating.columns]).show()

+-------+----+-----------+
|User-ID|ISBN|Book-Rating|
+-------+----+-----------+
|      0|   0|          0|
+-------+----+-----------+



### Items Table

In [ ]:
items = items.withColumn('Year-of-Publication', regexp_replace('Year-of-Publication', '-', ' ')).withColumnRenamed('Year-of-Publication', 'Publication Year')
df_items = items.select(['ISBN','Book-Title','Book-Author','Publication Year','Publisher'])
df_items.show()

+----------+--------------------+--------------------+----------------+--------------------+
|      ISBN|          Book-Title|         Book-Author|Publication Year|           Publisher|
+----------+--------------------+--------------------+----------------+--------------------+
|0195153448| Classical Mythology|  Mark P. O. Morford|            2002|Oxford University...|
|0002005018|        Clara Callan|Richard Bruce Wright|            2001|HarperFlamingo Ca...|
|0060973129|Decision in Normandy|        Carlo D'Este|            1991|     HarperPerennial|
|0374157065|Flu: The Story of...|    Gina Bari Kolata|            1999|Farrar Straus Giroux|
|0393045218|The Mummies of Ur...|     E. J. W. Barber|            1999|W. W. Norton &amp...|
|0399135782|The Kitchen God's...|             Amy Tan|            1991|    Putnam Pub Group|
|0425176428|What If?: The Wor...|       Robert Cowley|            2000|Berkley Publishin...|
|0671870432|     PLEADING GUILTY|         Scott Turow|            1993

In [ ]:
df_items.printSchema()

root
 |-- ISBN: string (nullable = true)
 |-- Book-Title: string (nullable = true)
 |-- Book-Author: string (nullable = true)
 |-- Publication Year: string (nullable = true)
 |-- Publisher: string (nullable = true)



Looking for NULL values in Items Table

In [ ]:
df_items.select([count(when(isnan(N) | col(N).isNull(), N)).alias(N) for N in df_items.columns]).show()

+----+----------+-----------+----------------+---------+
|ISBN|Book-Title|Book-Author|Publication Year|Publisher|
+----+----------+-----------+----------------+---------+
|   0|         0|        185|              25|        4|
+----+----------+-----------+----------------+---------+



Replacing NULL values with (0) Zero, only in the "Publication Year" column <br>
and 'UnKnown', in String datatype columns

In [ ]:
df_items = df_items.na.fill("0",["Publication Year"])
df_items = df_items.na.fill("unknown",["Book-Author"])
df_items = df_items.na.fill("unknown",["Publisher"]).show()

+----------+--------------------+--------------------+----------------+--------------------+
|      ISBN|          Book-Title|         Book-Author|Publication Year|           Publisher|
+----------+--------------------+--------------------+----------------+--------------------+
|0195153448| Classical Mythology|  Mark P. O. Morford|            2002|Oxford University...|
|0002005018|        Clara Callan|Richard Bruce Wright|            2001|HarperFlamingo Ca...|
|0060973129|Decision in Normandy|        Carlo D'Este|            1991|     HarperPerennial|
|0374157065|Flu: The Story of...|    Gina Bari Kolata|            1999|Farrar Straus Giroux|
|0393045218|The Mummies of Ur...|     E. J. W. Barber|            1999|W. W. Norton &amp...|
|0399135782|The Kitchen God's...|             Amy Tan|            1991|    Putnam Pub Group|
|0425176428|What If?: The Wor...|       Robert Cowley|            2000|Berkley Publishin...|
|0671870432|     PLEADING GUILTY|         Scott Turow|            1993

## Users Table

In [ ]:
df_users = users.select(['User-ID','Location','Age'])
df_users.show()

+-------+--------------------+----+
|User-ID|            Location| Age|
+-------+--------------------+----+
|      1|  nyc, new york, usa|null|
|      2|stockton, califor...|  18|
|      3|moscow, yukon ter...|null|
|      4|porto, v.n.gaia, ...|  17|
|      5|farnborough, hant...|null|
|      6|santa monica, cal...|  61|
|      7| washington, dc, usa|null|
|      8|timmins, ontario,...|null|
|      9|germantown, tenne...|null|
|     10|albacete, wiscons...|  26|
|     11|melbourne, victor...|  14|
|     12|fort bragg, calif...|null|
|     13|barcelona, barcel...|  26|
|     14|mediapolis, iowa,...|null|
|     15|calgary, alberta,...|null|
|     16|albuquerque, new ...|null|
|     17|chesapeake, virgi...|null|
|     18|rio de janeiro, r...|  25|
|     19|           weston, ,|  14|
|     20|langhorne, pennsy...|  19|
+-------+--------------------+----+
only showing top 20 rows



In [ ]:
df_users.printSchema()

root
 |-- User-ID: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Age: string (nullable = true)



In [ ]:
df_users.select([count(when(isnan(N) | col(N).isNull(), N)).alias(N) for N in df_users.columns]).show()

+-------+--------+------+
|User-ID|Location|   Age|
+-------+--------+------+
|      0|       1|110510|
+-------+--------+------+



Extracting the 'Location' column into 3 new columns:
1. City
2. Country
3. Region

In [ ]:
df_users = df_users.withColumn('City', split(df_users['Location'], ',').getItem(0))\
       .withColumn('Region', split(df_users['Location'], ',').getItem(1))\
       .withColumn('Country', split(df_users['Location'], ',').getItem(2))
df_users.show(truncate=False)

+-------+--------------------------------------+----+--------------+----------------+---------------+
|User-ID|Location                              |Age |City          |Region          |Country        |
+-------+--------------------------------------+----+--------------+----------------+---------------+
|1      |nyc, new york, usa                    |null|nyc           | new york       | usa           |
|2      |stockton, california, usa             |18  |stockton      | california     | usa           |
|3      |moscow, yukon territory, russia       |null|moscow        | yukon territory| russia        |
|4      |porto, v.n.gaia, portugal             |17  |porto         | v.n.gaia       | portugal      |
|5      |farnborough, hants, united kingdom    |null|farnborough   | hants          | united kingdom|
|6      |santa monica, california, usa         |61  |santa monica  | california     | usa           |
|7      |washington, dc, usa                   |null|washington    | dc           

In [ ]:
# Droping the column 'Location'
df_users = df_users.select(['User-ID','Age','City','Region','Country'])
df_users.show()

+-------+----+--------------+----------------+---------------+
|User-ID| Age|          City|          Region|        Country|
+-------+----+--------------+----------------+---------------+
|      1|null|           nyc|        new york|            usa|
|      2|  18|      stockton|      california|            usa|
|      3|null|        moscow| yukon territory|         russia|
|      4|  17|         porto|        v.n.gaia|       portugal|
|      5|null|   farnborough|           hants| united kingdom|
|      6|  61|  santa monica|      california|            usa|
|      7|null|    washington|              dc|            usa|
|      8|null|       timmins|         ontario|         canada|
|      9|null|    germantown|       tennessee|            usa|
|     10|  26|      albacete|       wisconsin|          spain|
|     11|  14|     melbourne|        victoria|      australia|
|     12|null|    fort bragg|      california|            usa|
|     13|  26|     barcelona|       barcelona|         

### Amir (Help needed)
we have an unresuvled issue that we could not fix by our selfs. 
<br>
After splitting the 'Location' column, 2 of the new columns have some NAN values in them (row 19 for example).
<br>
Bellow you can see 3 different approaches that i used in order to convert NAN values to string or NULL:
1. To Dellete all missing values
2. To Fill missing values with NULL or string (like "UnKnown")

The syntex is correct and the command is running, but the NAN values was not alttered/replaced. why? :-( 
<br>
That cause us another issue, since i cant expload the data to any type of file - becouse of that...I keep getting errors.


In [ ]:
df2=df_users.dropna(subset=['City|Region','Country'])
df2.show()

In [ ]:
df3 = df_users.na.fill("UnKnown",["Region"]).show()

+-------+----+--------------+----------------+---------------+
|User-ID| Age|          City|          Region|        Country|
+-------+----+--------------+----------------+---------------+
|      1|null|           nyc|        new york|            usa|
|      2|  18|      stockton|      california|            usa|
|      3|null|        moscow| yukon territory|         russia|
|      4|  17|         porto|        v.n.gaia|       portugal|
|      5|null|   farnborough|           hants| united kingdom|
|      6|  61|  santa monica|      california|            usa|
|      7|null|    washington|              dc|            usa|
|      8|null|       timmins|         ontario|         canada|
|      9|null|    germantown|       tennessee|            usa|
|     10|  26|      albacete|       wisconsin|          spain|
|     11|  14|     melbourne|        victoria|      australia|
|     12|null|    fort bragg|      california|            usa|
|     13|  26|     barcelona|       barcelona|         

In [ ]:
## Dosnt work :( tried to replace missing values with an 'UnKnown' string.
##df_users[['Region','Country']] = df_users[['Region','Country']].fillna('UnKnown') # Specific columns

## I get - TypeError: 'NoneType' object is not subscriptable

In [ ]:
df_users = df_users.na.fill("0",["Age"]).show()

+-------+---+--------------+----------------+---------------+
|User-ID|Age|          City|          Region|        Country|
+-------+---+--------------+----------------+---------------+
|      1|  0|           nyc|        new york|            usa|
|      2| 18|      stockton|      california|            usa|
|      3|  0|        moscow| yukon territory|         russia|
|      4| 17|         porto|        v.n.gaia|       portugal|
|      5|  0|   farnborough|           hants| united kingdom|
|      6| 61|  santa monica|      california|            usa|
|      7|  0|    washington|              dc|            usa|
|      8|  0|       timmins|         ontario|         canada|
|      9|  0|    germantown|       tennessee|            usa|
|     10| 26|      albacete|       wisconsin|          spain|
|     11| 14|     melbourne|        victoria|      australia|
|     12|  0|    fort bragg|      california|            usa|
|     13| 26|     barcelona|       barcelona|          spain|
|     14

# Data Analysis

In [ ]:
#averageRatings = df_items.groupBy('movieId').avg('rating')

In [ ]:
#topTen = averageRatings.orderBy('avg(rating)').take(10)

In [ ]:
df_j1 = items.join(ratings,items["ISBN"] == ratings["ISBN"]).show()

+----+----------+---+--------------------+---+--------------------+---+----------------+---+--------------------+---+--------------------+---+--------------------+---+--------------------+---+----+-------+----+----------+----+---+----+-----------+----+----+---+
| _c0|      ISBN| ;2|          Book-Title| ;4|         Book-Author| ;6|Publication Year| ;8|           Publisher|;10|         Image-URL-S|;12|         Image-URL-M|;14|         Image-URL-L|  "| _c0|User-ID| _c2|      ISBN| _c4|  ;| _c6|Book-Rating| _c8| _c9| ,,|
+----+----------+---+--------------------+---+--------------------+---+----------------+---+--------------------+---+--------------------+---+--------------------+---+--------------------+---+----+-------+----+----------+----+---+----+-----------+----+----+---+
|null|0002234947|  ;|Miss Hobbema Pageant|  ;|       W.P. Kinsella|  ;|            1990|  ;|Harpercollins Juv...|  ;|http://images.ama...|  ;|http://images.ama...|  ;|http://images.ama...|  "|null|  39608|null|0002

In [ ]:
items.show()

+----+----------+---+--------------------+---+--------------------+---+----------------+---+--------------------+---+--------------------+---+--------------------+---+--------------------+---+
| _c0|      ISBN| ;2|          Book-Title| ;4|         Book-Author| ;6|Publication Year| ;8|           Publisher|;10|         Image-URL-S|;12|         Image-URL-M|;14|         Image-URL-L|  "|
+----+----------+---+--------------------+---+--------------------+---+----------------+---+--------------------+---+--------------------+---+--------------------+---+--------------------+---+
|null|0195153448|  ;| Classical Mythology|  ;|  Mark P. O. Morford|  ;|            2002|  ;|Oxford University...|  ;|http://images.ama...|  ;|http://images.ama...|  ;|http://images.ama...|  "|
|null|0002005018|  ;|        Clara Callan|  ;|Richard Bruce Wright|  ;|            2001|  ;|HarperFlamingo Ca...|  ;|http://images.ama...|  ;|http://images.ama...|  ;|http://images.ama...|  "|
|null|0060973129|  ;|Decision in No

In [ ]:
items.groupBy("Publication Year").count().show()

# Export new dataset to CSV

In [ ]:
##df_users.repartition(1).write.format('com.databricks.spark.csv').save("/content/drive/MyDrive/BD Project/new_users.csv",header = 'true')


In [ ]:
##df_users.toPandas().to_csv('new_users.csv')

In [ ]:
spark.stop()

NameError: ignored